# [실습] 벡터 데이터베이스 기반 RAG 어플리케이션

RAG는 Retrieval-Augmented Generation (RAG) 의 약자로, 질문이 주어지면 관련 있는 문서를 찾아 프롬프트에 추가하는 방식의 어플리케이션입니다.   
RAG의 과정은 아래와 같이 진행됩니다.
1. Indexing : 문서를 받아 검색이 잘 되도록 저장합니다.
1. Processing : 입력 쿼리를 전처리하여 검색에 적절한 형태로 변환합니다<br>(여기서는 수행하지 않습니다)
1. Search(Retrieval) : 질문이 주어진 상황에서 가장 필요한 참고자료를 검색합니다.
1. Augmenting : Retrieval의 결과와 입력 프롬프트를 이용해 LLM에 전달할 프롬프트를 생성합니다.
1. Generation : LLM이 출력을 생성합니다.

In [1]:
!pip install --upgrade jsonlines openai langchain langchain-openai langchain-community beautifulsoup4 langchain_chroma chromadb==0.5.3

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/86/d5/b8378be0f4cf192992aa3080eb9ddcdca3109b399be61984424aaa79f847/openai-1.65.3-py3-none-any.whl.metadata
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/b5/d4/afe8174838bdd3baba5d6a19e9f3af4c54c5db1ab4d66ef0b650c6157919/langchain-0.3.20-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/2b/a3/6718deba2c30db991c6b000d23fa062441daa576eb1e520cb2edc2729e2f/langchain_community-0.3.19-py3-none-any.whl.metadata
  Obtaining dependency information for beautifulsoup4 from https://files.pythonhosted.org/packages/f9/49/6abb616eb3cbab6a7cca303dc02fdf3836de2e0b834bf966a7f5271a34d8/beautifulsoup4-4.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for langchain_chroma from https://files.pythonhosted.org/packages/e0/ad/02b43c9c6243d430a32fda148236ba02e49289326a58aeef57723

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("/Users/blueno/UNO/SKALA/SKALA/.env")
load_dotenv(dotenv_path=env_path, override=True)

# 환경 변수 확인
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ['USER_AGENT']='MyCustomAgent'
# 아래 코드 Warning 제거

## 1. `WebBaseLoader`로 웹 페이지 받아오기

LangChain의 `document_loaders`는 다양한 형식의 파일을 불러올 수 있습니다.   
[https://python.langchain.com/docs/integrations/document_loaders/ ]    

이번에는 웹 페이지를 로드하는 `WebBaseLoader`를 통해 뉴스 기사를 읽어보겠습니다.    
- 최근에는 FireCrawl(https://www.firecrawl.dev/)을 사용하는 경우가 늘고 있습니다.

네이버 API를 사용해, 네이버 뉴스 검색 링크를 가져옵니다.

In [3]:
import os
import requests

def get_naver_news_links(query, num_links=100):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={num_links}&sort=sim"
    # 최대 100개의 결과를 표시
    headers = {
        'X-Naver-Client-Id': os.getenv("NAVER_CLIENT_ID"),
        'X-Naver-Client-Secret': os.getenv("NAVER_CLIENT_SECRET")
    }

    response = requests.get(url, headers=headers)
    result = response.json()
    # 특정 링크 형식만 필터링
    filtered_links = []
    for item in result['items']:
        link = item['link']
        if "n.news.naver.com/mnews/article/" in link:
            # 네이버 뉴스 스타일만 모으기
            filtered_links.append(link)

    # 결과 출력
    print(len(filtered_links))
    for link in filtered_links:
        print(link)
    return filtered_links

filtered_links = []
for topic in ['LLM', '생성 인공지능', 'GPT', '딥러닝', '가전제품']:
    filtered_links += get_naver_news_links(topic, 100)
print(len(filtered_links))
print(len(list(set(filtered_links))))
filtered_links = list(set(filtered_links))

54
https://n.news.naver.com/mnews/article/293/0000064488?sid=105
https://n.news.naver.com/mnews/article/037/0000035830?sid=105
https://n.news.naver.com/mnews/article/277/0005554480?sid=105
https://n.news.naver.com/mnews/article/015/0005101683?sid=105
https://n.news.naver.com/mnews/article/001/0015243264?sid=105
https://n.news.naver.com/mnews/article/003/0013096548?sid=105
https://n.news.naver.com/mnews/article/014/0005316599?sid=101
https://n.news.naver.com/mnews/article/293/0000064445?sid=105
https://n.news.naver.com/mnews/article/003/0013097927?sid=105
https://n.news.naver.com/mnews/article/031/0000912965?sid=105
https://n.news.naver.com/mnews/article/029/0002939041?sid=102
https://n.news.naver.com/mnews/article/417/0001061403?sid=105
https://n.news.naver.com/mnews/article/030/0003289256?sid=105
https://n.news.naver.com/mnews/article/018/0005954203?sid=105
https://n.news.naver.com/mnews/article/277/0005555414?sid=101
https://n.news.naver.com/mnews/article/018/0005955199?sid=105
https

WebBaseLoader를 이용해, 링크로부터 본문을 불러옵니다.

In [4]:
# 주의: 동일 IP 환경에서 동시에 다수 실행하면 차단의 위험이 있음

import bs4
from langchain_community.document_loaders import WebBaseLoader
def get_news_documents(links):
    loader = WebBaseLoader(
        web_paths=links,
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("newsct", "newsct-body")
                # newsct, newsct-body만 추출 : 도메인마다 다름
            )
        ),
        requests_per_second = 1, # 1초에 1개 요청 보내기
        show_progress = True # 진행 상황 출력
    )
    docs = loader.load()
    len(docs)
    return docs
docs = get_news_documents(filtered_links)


In [5]:
print(docs[0:4])

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/023/0003891321?sid=105'}, page_content='\n\n\n\n\n조선일보\n\n조선일보\n\n\n구독\n\n조선일보 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n조선일보 언론사 구독 해지되었습니다.\n\n\n\n\n“GPT-4.5 거대하고 비싼 모델”… 올트먼도 비판적 평가\n\n\n\n\n입력2025.03.04. 오전 1:31\n\n\n수정2025.03.04. 오전 3:37\n\n기사원문\n \n\n\n\n\n오로라 기자\n\n\n\n\n\n\n\n\n오로라 기자\n\n\n\n\n오로라 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\nGPU 부족, 

In [6]:
# 참고) 불러온 document 저장하기

import jsonlines
def save_docs_to_jsonl(documents, file_path):
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())
save_docs_to_jsonl(docs, "docs.jsonl")

# 참고) jsonl 파일 불러오기
from langchain.schema import Document

def load_docs_from_jsonl(file_path):
    documents = []
    with jsonlines.open(file_path, mode="r") as reader:
        for doc in reader:
            documents.append(Document(**doc))
    return documents


/var/folders/fn/7bjmjvmn2d30b47j1yfz_5jm0000gn/T/ipykernel_72024/3672484833.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  writer.write(doc.dict())


In [7]:
# 파일로부터 다시 불러오기
docs = load_docs_from_jsonl("docs.jsonl")
len(docs)

276

In [8]:
docs[0].page_content

'\n\n\n\n\n조선일보\n\n조선일보\n\n\n구독\n\n조선일보 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n조선일보 언론사 구독 해지되었습니다.\n\n\n\n\n“GPT-4.5 거대하고 비싼 모델”… 올트먼도 비판적 평가\n\n\n\n\n입력2025.03.04. 오전 1:31\n\n\n수정2025.03.04. 오전 3:37\n\n기사원문\n \n\n\n\n\n오로라 기자\n\n\n\n\n\n\n\n\n오로라 기자\n\n\n\n\n오로라 기자\n\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\nGPU 부족, 高價 구독자만 서비스최신 모델인데 성능 기대 못 미쳐\t\t\t\t\t\t\t\t\t\t오픈AI가 지난달 27일 내놓은 최신 인공지능(AI) 모델 ‘GPT-4.5’에 대해 내부적으로 비판적인 평가

불러온 뉴스기사에는 불필요한 내용이 다소 존재합니다.    
아래의 과정을 통해 간단하게 전처리합니다.

In [9]:
def preprocess(docs):
    noise_texts = [
        '''구독중 구독자 0 응원수 0 더보기''',
        '''쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0''',
        '''댓글 본문 요약봇 본문 요약봇''',
        '''도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기''',
        '''텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름''',
        '''이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작''',
        '''닫기 글자 크기 변경하기 가1단계 작게 가2단계 보통 가3단계 크게 가4단계 아주크게 가5단계 최대크게 SNS 보내기 인쇄하기''',


    ]

    def clean_text(text):
        # 잡음을 제거하고 여러 공백을 하나로 줄이는 함수
        text = text.replace('\t',' ').replace('\n',' ')
        for _ in range(20):
            text = ' '.join(text.split())  # 연속된 공백을 하나로
        for noise in noise_texts:
            text = text.replace(noise, '')

        return text

    preprocessed_docs = []
    for doc in docs:
        try:
            # 텍스트 자르기
            content = doc.page_content.split('구독 해지되었습니다.')[1]
        except:
            # 구독 관련 정보가 없거나 텍스트 처리 문제일 경우 원본 텍스트 유지
            content = doc.page_content
        try:
            content = doc.page_content.split('구독 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!')[0]
        except:
            content = doc.page_content


        # 텍스트 정제 작업
        content = clean_text(content)
        doc.page_content = content
        preprocessed_docs.append(doc)

    return preprocessed_docs

preprocessed_docs = preprocess(docs)


In [10]:
preprocessed_docs[10]


Document(metadata={'source': 'https://n.news.naver.com/mnews/article/374/0000427835?sid=101'}, page_content='SBS Biz SBS Biz 구독 SBS Biz 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 SBS Biz 언론사 구독 해지되었습니다. 유영상 SKT 대표 "AI 데이터센터 토털 솔루션 시장 공략" 입력2025.03.03. 오전 8:00 기사원문 조슬기 기자 조슬기 기자 조슬기 기자 구독  추천      에이닷 비즈 연내 출시…한국어 특화 LLM 상반기 개발 완료 유영상 SK텔레콤 대표이사(CEO)가 향후 SK그룹 멤버사와 국내외 파트너사의 최신 기술을 결집해 AI 데이터센터(AI DC) 토털 솔루션 시장 공략에 나설 것이라고 밝혔습니다.이를 위해 GPU 렌탈부터 소규모 모듈러(Modular), 고객 맞춤형 전용(Dedicated), 하이퍼스케일 급까지, 모든 유형의 AI DC 수요를 충족하는 솔루션 사업으로 \'AI 인프라 슈퍼 하이웨이(AI Infra Super Highway)\' 구축을 가속화한다는 전략입니다.유 CEO는 현지시간 2일, 세계 최대 이동통신 전시회인 \'모바일 월드 콩그레스(MWC) 2025\' 행사가 열리는 스페인 바르셀로나에서 기자간담회를 개최하고 AI DC 사업 모델 및 AI 에이전트 B2B·B2C 고도화 전략, 자강과 협력을 통한 AI 기술 경쟁력 강화에 대한 청사진이 담긴 AI 피라미드 2.0 전략을 공개했습니다.유 CEO는 "AI DC 사업은 \'알라카르테(a la carte; 맞춤형 상품)\' 형태를 갖춰 모든 유형의 고객 수요를 충족할 수 있도록 고도화하고, AI 에이전트 B2B와 B2C도 고객에게 새로운 경험을 지속 제공하는 서비스 혁신과 함께 자강과 협력 기반으로 AI 테크 역량도 지속 강화해 AI 매출 성장을 이루겠다"라고 말했습니다.AI DC 4대 사업 모델 라인업 구축 시장 공략 SKT는 우선

## 2. Chunking: 청크 단위로 나누기   



전처리가 완료된 docs를 chunk 단위로 분리합니다.
`chunk_size`와 `chunk_overlap`을 이용해 청크의 구성 방식을 조절할 수 있습니다.

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
# 0~2000, 1800~3800, 3600~5600, ...
chunks = text_splitter.split_documents(preprocessed_docs)
print(len(chunks))

551


구성된 청크를 벡터 데이터베이스에 로드합니다.   
`Chroma.from_documents`는 documents의 임베딩을 구하고 이를 DB에 저장합니다.

In [12]:
chunks[110].page_content

'아시아경제 아시아경제 구독 아시아경제 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 아시아경제 언론사 구독 해지되었습니다. 제이엘케이, 응급 뇌출혈 진단 혁신…딥러닝 특허로 글로벌 경쟁력 입증 입력2025.02.18. 오전 8:51 수정2025.02.18. 오전 8:52 기사원문 장효원 기자 장효원 기자 장효원 기자 구독  추천      의료 AI 1호 상장기업 제이엘케이(대표 김동민)는 회사의 뇌졸중 AI 솔루션에 활용되는 딥러닝 기반 뇌출혈 진단장치 및 방법에 대한 국내 기술 특허 취득에 성공했다고 18일 밝혔다.제이엘케이가 출원한 ‘딥러닝 기반 뇌출혈 진단 장치 및 방법’은 이달 초 최종 등록이 결정됐다. 제이엘케이는 기존 3차원 의료 영상의 분석 및 진단 속도가 느려 응급 환자에게 적용하기 어려웠던 문제에 주목해 이번 특허 기술을 확보한 것으로 알려졌다.해당 특허는 복수의 CT 영상을 딥러닝 모델로 분석해 뇌출혈 병변을 검출하고 3차원으로 재구축해 환자의 뇌출혈 확률을 신속하게 산출할 수 있는 것이 특징이다. MRI보다 저렴하고 보다 신속히 촬영할 수 있는 CT 영상만으로도 뇌출혈을 정확히 진단할 수 있어 응급 상황에서의 빠른 치료 계획 수립에 필요한 기술이다.뇌졸중 의심 환자의 병원 이송 시 의료진이 가장 먼저 진행하는 진단 과정이 비조영 CT 촬영을 통한 뇌출혈 여부인 점을 감안할 때, 이번 특허 기술이 응급 의료 현장에서의 진단 혁신을 이루는 중요한 역할을 할 것으로 예상된다.특히 세계 최대 의료시장으로 꼽히는 미국과 빠른 고령화 속도를 보이고 있는 일본의 경우 뇌졸중 AI 솔루션의 수요가 높을 것으로 예상되는 만큼 원천 기술의 역량이 입증된 해당 솔루션에 대한 수요도 충분할 것으로 기대된다.제이엘케이 김동민 대표는 “딥러닝 기반 뇌출혈 진단 장치 및 방법에 대한 국내 기술 특허 취득으로 제이엘케이의 의료 AI 기술 역량을 대내외적으로 인정받고 기술 방어력을 높일 수 있게 됐다”며 “미국과 일본을 포함한 글로벌 시장

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [14]:
Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제

# 이미 존재하는 DB에서 가져오기
# db = Chroma(
#      embedding=OpenAIEmbeddings(model='text-embedding-3-large'),
#      large: 3072차원, small: 1536차원
#      persist_directory="./chroma_Web"
#      )
# Collection 이름 없이 생성한 경우, 기본 Collection으로 저장된 경우

db = Chroma.from_documents(documents=chunks,
                           embedding=OpenAIEmbeddings(model='text-embedding-3-small'), # text-embedding-3-large
                           # large: 3072차원, small: 1536차원
                           persist_directory="./chroma_Web", # ./는 current directory를 의미
                           # persist_directory를 쓰지 않으면 메모리에 저장
                           collection_metadata={'hnsw:space':'l2'}
                           # l2 메트릭 설정(기본값)
                           # cosine, mmr
                           )
# Collection 이름 없이 생성한 경우, 기본 Collection으로 저장됨

retriever는 query에 맞춰 db에서 문서를 검색합니다.

In [15]:
retriever = db.as_retriever()

In [16]:
retriever.invoke("도메인 특화 LLM에 대한 소식 있어?")

[Document(id='03abac04-8daf-4fbe-b0d2-67ac961361b8', metadata={'source': 'https://n.news.naver.com/mnews/article/003/0013084396?sid=102'}, page_content='뉴시스 뉴시스 구독 뉴시스 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 뉴시스 언론사 구독 해지되었습니다. PICK 안내 언론사가 주요기사로선정한 기사입니다. 언론사별 바로가기 닫기 챗GPT에 손쉽게 개인정보 뚫린다…KAIST, LLM 악용 가능성 입증 입력2025.02.24. 오후 1:53 수정2025.02.24. 오후 2:27 기사원문 김양수 기자 김양수 기자 김양수 기자 구독  추천      신승원·이기민 교수팀, 대형언어모델의 정보수집·피싱 공격 규명최대 95.9% 정확도로 개인정보 수집, 피싱 이메일 클릭률 46.67%까지 증가저비용으로 개인정보 탈취해 공격 가능…국제학술지 게재 [대전=뉴시스] KAIST가 챗GPT 등 대형언어모델(LLM)을 활용한 개인정보 수집 및 피싱공격 실험을 수행해 LLM 에이전트가 손쉽게 개인정보 수집 및 피싱공격 등을 할 수 있다는 사실을 입증했다.(윗줄 왼쪽부터)KAIST 전기및전자공학부 나승호 박사, 김재철AI대학원 이기민 교수.(아랫줄 왼쪽부터)전기및전자공학부 김한나 박사과정, 신승원 교수, 송민규 박사과정.(사진=KAIST 제공) *재판매 및 DB 금지[대전=뉴시스] 김양수 기자 = 챗GPT로 손쉽게 개인정보를 수집하고 악용할 수 있다는 연구결과가 국내연구진에 의해 발표됐다.한국과학기술원(KAIST)은 전기및전자공학부 신승원 교수, 김재철 AI 대학원 이기민 교수 공동연구팀이 실제환경에서 챗GPT와 같은 대형 언어모델(LLM)이 사이버공격에 악용될 가능성을 실험적으로 규명했다고 24일 밝혔다.최근 인공지능 기술의 발전으로 LLM은 단순한 챗봇을 넘어 자율적인 에이전트로 발전하고 있다. 현재 OpenAI, 구글 AI 

## 3. Prompting

RAG를 위한 간단한 프롬프트를 작성합니다.

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
최소 3문장에서 최대 5문장으로 답변하고, 정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])
prompt.pretty_print() # pretty_print() 사람이 보기 편하게 출력

================================ Human Message =================================

당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
최소 3문장에서 최대 5문장으로 답변하고, 정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면, "정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


## 4. Chain

RAG를 수행하기 위한 Chain을 만듭니다.

RAG Chain은 프롬프트에 context와 question을 전달해야 합니다.    
체인의 입력은 Question만 들어가므로, Context를 동시에 prompt에 넣기 위해서는 아래의 구성이 필요합니다.

In [19]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)


# retriever의 결과물은 List[Document] 이므로 이를 ---로 구분하는 함수
# metadata의 source를 보존하여 추가
def format_docs(docs):
    return "\n\n---\n\n".join([doc.page_content+ '\nURL: '+ doc.metadata['source'] for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결
    # Ex)'와'.join(['a','b','c']) = 'a와b와c'

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # retriever : question을 받아서 context 검색: document 반환
    # format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
rag_chain.invoke("도메인 특화 LLM이 뭔가요?")

'도메인 특화 LLM(대형 언어 모델)은 특정 분야나 산업에 최적화된 인공지능 언어 모델을 의미합니다. 이러한 모델은 일반적인 LLM과 달리 특정 조직이나 기업의 요구에 맞춰 설계되어, 해당 분야의 전문 데이터를 학습하여 보다 정교하고 신뢰성 높은 정보를 생성할 수 있습니다. 예를 들어, 금융 산업에서는 복잡한 용어와 개념을 다루기 때문에, 금융 관련 데이터를 기반으로 학습한 도메인 특화 LLM이 더욱 효과적으로 활용될 수 있습니다. 이러한 모델은 보안성이 뛰어나고, 특정 분야의 전문성을 강화하는 데 큰 도움이 됩니다.'

In [21]:
rag_chain.invoke("오픈 AI 최근 소식 있어?")

"오픈AI는 최근 'GPT-4.5'라는 새로운 인공지능 모델을 공개했습니다. 이 모델은 추론 기능이 없는 마지막 일반 모델로, 특정 사용자 그룹에 제한적으로 제공되며, 월 200달러의 '챗GPT 프로' 요금제 사용자에게 우선 제공됩니다. GPT-4.5는 이전 모델들보다 더 높은 정확도와 감성지능을 갖추고 있어, 보다 자연스러운 대화가 가능하다고 평가받고 있습니다. 오픈AI는 향후 GPT-5에서 추론 모델과 일반 모델을 통합할 계획이라고 밝혔습니다."

In [22]:
rag_chain.invoke("한국에서 만든 초거대 언어 모델은 뭐가 있나요? 참고 링크도 올려주세요")

'정보가 부족하여 답변할 수 없습니다.'

In [23]:
rag_chain.invoke("sllm이 뭐야?")

'정보가 부족하여 답변할 수 없습니다.'

만약 Context가 포함된 RAG 결과를 보고 싶다면, RunnableParallel을 사용하면 됩니다.

assign()을 이용하면, 체인의 결과를 받아 새로운 체인에 전달하고, 그 결과를 가져옵니다.

In [24]:
from langchain_core.runnables import RunnableParallel
# assign : 결과를 받아서 새로운 인수 추가하고 원래 결과와 함께 전달

rag_chain_from_docs = (
    prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("엔비디아에 대한 소식은?")

# retriever가 1번 실행됨
# retriever의 실행 결과를 rag_chain_from_docs 에 넘겨주기 때문에

{'context': '들었다놨다" 한국인들 갇혔다…캄보디아서 무슨 일 홀로 묵던 투숙객 숨져…함께 갯벌 간 남편 사망, 아내 실종 기상예보 믿고 앞당겼다 텅…"못 견디고 소멸" 결국 접었다 "싹 없어졌다" 뜻밖의 흉물…10층 올리다 올스탑 "줄도산?" 옥중편지 대독에 "콕 집어 저격"…경찰출신 의원도 거친 입 이 기사를 추천합니다 기사 추천은 24시간 내 50회까지 참여할 수 있습니다. 닫기  모두에게 보여주고 싶은 기사라면?beta 이 기사를 추천합니다 버튼을 눌러주세요. 집계 기간 동안 추천을 많이 받은 기사는 네이버 자동 기사배열 영역에 추천 요소로 활용됩니다. 레이어 닫기 SBS 언론사홈 바로가기 기자 구독 후 기사보기 구독 없이 계속 보기\nURL: https://n.news.naver.com/mnews/article/055/0001236444?sid=100\n\n---\n\nSBS의 구독 많은 기자를 구독해보세요! 닫기 Copyright ⓒ SBS. All rights reserved. 무단 전재, 재배포 및 AI학습 이용 금지 이 기사는 언론사에서 생활 섹션으로 분류했습니다. 기사 섹션 분류 안내 기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다. 언론사는 개별 기사를 2개 이상 섹션으로 중복 분류할 수 있습니다. 닫기 구독 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요! 구독중 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 확인해보세요! 네이버에서 SBS뉴스를 구독해주세요! 가장 확실한 SBS 제보 [클릭!] SBS SBS 주요뉴스해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다. "발톱 들었다놨다" 한국인들 갇혔다…캄보디아서 무슨 일 홀로 묵던 투숙객 숨져…함께 갯벌 간 남편 사망, 아내 실종 기상예보 믿고 앞당겼다 텅…"못 견디고 소멸" 결국 접었다 "싹 없어졌다" 뜻밖의 흉물…10층 올리다 올스탑 "줄도산?" 옥중편지 대독에 "콕 집어 저격"…경찰출신 의원도 거친 입 이 기사를 추천합니다 기

In [25]:
# Runnable Quiz

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})
# 결과 이해해보기!

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

## Gradio로 배포하기

입력과 출력을 통해 RAG를 수행합니다.   
크롤링을 수행하는 것은 시간이 오래 걸리기 때문에,     
기본값을 5로 두고 수정 가능하게 만들어 보겠습니다.

In [26]:
def construct_vector_db(query, num_links=5):
    filtered_links = get_naver_news_links(query, num_links)
    docs = get_news_documents(filtered_links)
    preprocessed_docs = preprocess(docs)
    chunks = text_splitter.split_documents(preprocessed_docs)
    Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제

    db = Chroma.from_documents(documents=chunks,
                               embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
                               collection_metadata={'hnsw:space':'l2'})
    return db

def RAG(query, question, num_links=5):
    db = construct_vector_db(query, num_links)
    retriever = db.as_retriever()
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain.invoke(question)


In [33]:
!pip install gradio

  Obtaining dependency information for markupsafe~=2.0 from https://files.pythonhosted.org/packages/11/e7/291e55127bb2ae67c64d66cef01432b5933859dfb7d6949daa721b89d0b3/MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl.metadata
  Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.0 kB)
Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl (18 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langflow 1.0.18 requires certifi<2025.0.0,>=2023.11.17, but you have certifi 2025.1.31 which is incompatible.
langflow 1.0.18 requires chromadb<0.5,>=0.4, but you have chromadb 0.5.3 which is incompatible.
langflow 1.0.18 requires crewai[tools]<0.37.0,>=0.36.

In [27]:
import gradio as gr

def gradio_rag_interface(query, question, num_links):
    try:
        response = RAG(query, question, num_links)
        return response
    except Exception as e:
        return f"에러 발생: {str(e)}"

with gr.Blocks() as demo:
    gr.Markdown("# 🔍 네이버 뉴스용 검색 증강 생성(RAG)")
    gr.Markdown(
        """
        네이버 뉴스와 관련된 쿼리를 입력하면, 시스템이 관련 기사를 검색,
        처리하고 검색된 정보를 기반으로 종합적인 답변을 생성합니다.
        """
    )

    with gr.Row():
        query_input = gr.Textbox(
            label="📄 쿼리 입력",
            placeholder="예: 최신 AI 기술 발전",
            lines=2
        )
        num_links_input = gr.Slider(
            minimum=1,
            maximum=20,
            step=1,
            value=5,
            label="🔗 뉴스 링크 수",
            info="검색 및 처리할 뉴스 기사의 수를 선택하세요."
        )
        question_input = gr.Textbox(
            label="❓ 질문 입력",
            placeholder="예: 퓨리오사 AI는 어떤 일을 했나요?",
            lines=2
        )

    submit_button = gr.Button("✅ 답변 생성")

    output_box = gr.Textbox(
        label="📝 생성된 답변",
        placeholder="답변이 여기에 표시됩니다...",
        lines=10
    )

    # 버튼 클릭 이벤트 정의
    submit_button.click(
        fn=gradio_rag_interface,
        inputs=[query_input, question_input, num_links_input],
        outputs=output_box
    )

    gr.Markdown(
        """
        ---
        [Gradio](https://gradio.app/) 및 OpenAI의 GPT 모델을 사용하여 제작되었습니다.
        """
    )


demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


4
https://n.news.naver.com/mnews/article/032/0003354348?sid=104
https://n.news.naver.com/mnews/article/417/0001061918?sid=101
https://n.news.naver.com/mnews/article/421/0008097859?sid=104
https://n.news.naver.com/mnews/article/032/0003354246?sid=104
7
https://n.news.naver.com/mnews/article/032/0003354348?sid=104
https://n.news.naver.com/mnews/article/417/0001061918?sid=101
https://n.news.naver.com/mnews/article/421/0008097859?sid=104
https://n.news.naver.com/mnews/article/032/0003354246?sid=104
https://n.news.naver.com/mnews/article/215/0001200493?sid=004
https://n.news.naver.com/mnews/article/009/0005453075?sid=101
https://n.news.naver.com/mnews/article/003/0013078167?sid=101
7
https://n.news.naver.com/mnews/article/032/0003354348?sid=104
https://n.news.naver.com/mnews/article/417/0001061918?sid=101
https://n.news.naver.com/mnews/article/421/0008097859?sid=104
https://n.news.naver.com/mnews/article/032/0003354246?sid=104
https://n.news.naver.com/mnews/article/215/0001200493?sid=004
ht